<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/binder_hallucination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - binder hallucination
For a given protein target and protein binder length, generate/hallucinate a protein binder sequence AlphaFold thinks will bind to the target structure. To do this, we maximize number of contacts at the interface and maximize pLDDT of the binder.

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold).

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/ColabDesign.git
  ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params
fi

In [ ]:
#@title import libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.shared.utils import copy_dict
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
#@title prep inputs
import re
pdb = "4N5T" #@param {type:"string"}
chain = "A" #@param {type:"string"}
binder_len =  15#@param {type:"integer"}
binder_seq = "" #@param {type:"string"}
binder_seq = re.sub("[^A-Z]", "", binder_seq.upper())

if len(binder_seq) > 0:
  binder_len = len(binder_seq)
hotspot = "" #@param {type:"string"}
#@markdown ---
#@markdown model config
use_multimer = False #@param {type:"boolean"}
if use_multimer:
  num_recycles = 1
  iters = 100
else:
  num_recycles = 0
  iters = 100

if hotspot == "":
  hotspot = None

x = {"pdb_filename":pdb,
     "chain":chain,
     "binder_len":binder_len,
     "hotspot":hotspot,
     "use_multimer":use_multimer}
x["pdb_filename"] = get_pdb(x["pdb_filename"])     

if "x_prev" not in dir() or x != x_prev:
  clear_mem()
  model = mk_afdesign_model(protocol="binder",
                            use_multimer=x["use_multimer"])
  model.prep_inputs(**x)
  x_prev = copy_dict(x)
  print("target length:", model._target_len)
  print("binder length:", model._binder_len)

In [ ]:
#@title **run AfDesign**
optimizer = "3stage" #@param ["3stage", "semigreedy"]
weights = {"con":0.0,
           "pae":0.0,
           "plddt":0.1,
           "i_pae":0.0,
           "i_con":1.0}
opt = {
    "con":{"seqsep":9, "cutoff":14.0, "num":1, "binary":False},
    "i_con":{"cutoff":21.6875, "num":1, "num_pos":float("inf"), "binary":False}
}

model.restart(opt=opt, weights=weights)
if len(binder_seq) > 1: model.set_seq(seq=binder_seq)
if optimizer == "3stage":
  model.design_3stage(iters, 100, 10, num_recycles=num_recycles, verbose=True)
else:
  model.design_semigreedy(25, num_models=1, num_recycles=num_recycles, verbose=True)

In [ ]:
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb(animate=True)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()